**Заказчик: Благотворительный фонд “Бюро добрых дел”**

Основная цель заказчика заключается в том, чтобы из пула тех клиентов, которые жертвовали за последний год, выявить клиентов, которые могут увеличить частоту или размер пожертвований.

Данные, которыми располагает заказчик, включают в себя историю платежей жертвователей и какую-то дополнительную информацию за несколько лет по четырём тысячам жертвователей.

Жизненный цикл жертвователя составляет от полутора до трёх лет, поэтому в истории будут примеры того, как именно меняется характер платежей клиентов.

Если получится на основе данных сделать дополнительную сегментацию, например выделить характерное поведение пользователей, которые прекращают жертвовать, это также интересует заказчика.

In [213]:
%pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [214]:
%pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
# импортируем необходимые библиотеки 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # для масштабирования количественных признаков
from sklearn.model_selection import GridSearchCV # для поиска оптимальных параметров

from sklearn.metrics import make_scorer
from scipy import stats as st # для доверительного интервала
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve 
import time


#доп настройки
pd.options.display.expand_frame_repr = False
import warnings
warnings.filterwarnings('ignore')  

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
donor = pd.read_excel('/content/drive/My Drive/Colab Notebooks/masterskaya/donor.xlsx')
donat = pd.read_excel('/content/drive/My Drive/Colab Notebooks/masterskaya/donations.xlsx')

In [131]:
# открываем и смотрим файлы
def info_df (donar, donat):
  df_all = [donor, donat]
  for i in df_all:
    display(i.info())
    display(i.head())
    display(i.isna().sum())

display(info_df (donor, donat))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   ID                               3683 non-null   object        
 1   Пол                              2431 non-null   object        
 2   Город                            1015 non-null   object        
 3   Дата последней email активности  1835 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 115.2+ KB


None

,ID,Пол,Город,Дата последней email активности
0,AF575B85-917F-475D-8875-00055586E05B,NaN,NaN,NaT
1,229A9B43-3A15-4A48-8CB6-000EDB240D50,NaN,NaN,NaT
2,C6606486-05A8-4437-ACF9-00344E54FEE1,NaN,NaN,NaT
3,6ED86154-A688-4A11-868F-006209035413,Женский ...,NaN,NaT
4,46C41E20-EA56-4698-9C37-00692FD6FFE8,Женский ...,NaN,NaT


ID                                    0
Пол                                1252
Город                              2668
Дата последней email активности    1848
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14043 entries, 0 to 14042
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Donor ID            14043 non-null  object        
 1   Дата платежа        14043 non-null  datetime64[ns]
 2   Сумма платежа       14043 non-null  float64       
 3   Рекуррентный        14043 non-null  object        
 4   Назначение платежа  12774 non-null  object        
 5   Канал оплаты        14043 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 658.4+ KB


None

,Donor ID,Дата платежа,Сумма платежа,Рекуррентный,Назначение платежа,Канал оплаты
0,DE5393F9-9585-4057-B7FB-24B69697FC08,2022-07-03 19:49:48,300.0,Да ...,ПОЖЕРТВОВАНИЕ undefined,Банковские карты
1,81CD4BEF-443E-4D36-B34F-664E11182D65,2020-03-14 12:20:02,300.0,Нет ...,SMS bdd,SMS
2,D3BEB76D-D8B6-47E2-8109-44F1B278943B,2022-04-07 00:00:00,16.0,Нет ...,Добровольное пожертвование. НДС не облагается,Прямые поступления на счет
3,F1310736-B229-4AAC-99B7-F13ACECF67CB,2019-12-26 02:25:42,300.0,Нет ...,NaN,SMS
4,61C3D86B-39DA-445C-9D7F-9924CE9BDA14,2021-06-26 01:26:07,200.0,Да ...,ПОЖЕРТВОВАНИЕ undefined,Банковские карты


Donor ID                 0
Дата платежа             0
Сумма платежа            0
Рекуррентный             0
Назначение платежа    1269
Канал оплаты             0
dtype: int64

None

Видно много пропусков, неизвестных значений, которые либо удалить , либо заменить. 

In [132]:
# посмотрим процент пропусков
display(donor.isnull().sum() * 100 / len(donor))
display(donat.isnull().sum() * 100 / len(donat))

ID                                  0.000000
Пол                                33.994027
Город                              72.440945
Дата последней email активности    50.176487
dtype: float64

Donor ID              0.000000
Дата платежа          0.000000
Сумма платежа         0.000000
Рекуррентный          0.000000
Назначение платежа    9.036531
Канал оплаты          0.000000
dtype: float64

Больше всего пропусков в Городе, поле и дате активности по почте.

In [133]:
# проверим на дубликаты
donor.duplicated().sum()

0

In [134]:
# проверим на дубликаты
donat.duplicated().sum()

12

In [135]:
# удаляем дубликаты и проверяем
donat = donat.drop_duplicates()
donat.duplicated().sum()

0

Удалили дубликаты.

## Предобработка данных

In [136]:
# приведём столбцы к змеиному регистру
donor = donor.rename(columns={'ID':'id',
                              'Пол':'sex',
                              'Город':'city',
                              'Дата последней email активности':'e-mail_act'})

donor

,id,sex,city,e-mail_act
0,AF575B85-917F-475D-8875-00055586E05B,NaN,NaN,NaT
1,229A9B43-3A15-4A48-8CB6-000EDB240D50,NaN,NaN,NaT
2,C6606486-05A8-4437-ACF9-00344E54FEE1,NaN,NaN,NaT
3,6ED86154-A688-4A11-868F-006209035413,Женский ...,NaN,NaT
4,46C41E20-EA56-4698-9C37-00692FD6FFE8,Женский ...,NaN,NaT
...,...,...,...,...
3678,C5E55606-9BFC-48C0-BE7F-FFC10A10DF20,Женский ...,NaN,NaT
3679,3CB754A9-7B08-40FB-A350-FFDB291DA9E1,NaN,NaN,NaT
3680,3F073C3B-E567-428E-B16F-FFE09A8D2CE0,Женский ...,NaN,2022-07-18 16:29:34
3681,029C3CEE-2381-4050-8E91-FFE3030218D4,Женский ...,КУШВА,NaT


In [137]:
donat = donat.rename(columns={'Donor ID':'id',
                              'Дата платежа':'pay_date',
                              'Сумма платежа':'pay_sum',
                              'Рекуррентный':'recurrent',
                              'Назначение платежа':'purpose_of_payment',
                              'Канал оплаты':'payment_channel'})

donat

,id,pay_date,pay_sum,recurrent,purpose_of_payment,payment_channel
0,DE5393F9-9585-4057-B7FB-24B69697FC08,2022-07-03 19:49:48,300.0,Да ...,ПОЖЕРТВОВАНИЕ undefined,Банковские карты
1,81CD4BEF-443E-4D36-B34F-664E11182D65,2020-03-14 12:20:02,300.0,Нет ...,SMS bdd,SMS
2,D3BEB76D-D8B6-47E2-8109-44F1B278943B,2022-04-07 00:00:00,16.0,Нет ...,Добровольное пожертвование. НДС не облагается,Прямые поступления на счет
3,F1310736-B229-4AAC-99B7-F13ACECF67CB,2019-12-26 02:25:42,300.0,Нет ...,NaN,SMS
4,61C3D86B-39DA-445C-9D7F-9924CE9BDA14,2021-06-26 01:26:07,200.0,Да ...,ПОЖЕРТВОВАНИЕ undefined,Банковские карты
...,...,...,...,...,...,...
14038,0B6DA68C-558B-4153-806E-B1D775585B88,2015-12-12 14:50:34,100.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты
14039,793690C0-6332-4289-96B4-2CF8902ACEF5,2020-12-02 09:59:13,200.0,Да ...,ПОЖЕРТВОВАНИЕ,Банковские карты
14040,69613503-5B71-4690-ACB5-8E783FF915B4,2021-12-26 00:00:00,500.0,Нет ...,Добровольное пожертвование. НДС не облагается,Прямые поступления на счет
14041,144B2E46-2A6A-48DE-92FB-729C3897A2B4,2018-11-27 16:35:01,200.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты


Посмотрим сколько уникальных айди, возможно подойдёт для объединения таблиц.

In [138]:
len(donor['id'].unique())

3683

In [139]:
len(donat['id'].unique())

3683

Кол-во уникальных значений одинаковое, поэтому соединять таблицы будем по этому столбцу.

In [140]:
df = donor.merge(donat, how='left', on='id')
df.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment,payment_channel
0,AF575B85-917F-475D-8875-00055586E05B,NaN,NaN,NaT,2019-07-06 15:23:53,100.0,Нет ...,NaN,SMS
1,229A9B43-3A15-4A48-8CB6-000EDB240D50,NaN,NaN,NaT,2017-07-18 09:01:24,500.0,Нет ...,NaN,SMS
2,C6606486-05A8-4437-ACF9-00344E54FEE1,NaN,NaN,NaT,2019-10-21 18:31:00,300.0,Нет ...,NaN,SMS
3,6ED86154-A688-4A11-868F-006209035413,Женский ...,NaN,NaT,2021-04-13 00:00:00,500.0,Нет ...,Добровольное пожертвование. НДС не облагается,Прямые поступления на счет
4,46C41E20-EA56-4698-9C37-00692FD6FFE8,Женский ...,NaN,NaT,2019-09-10 15:50:55,1042.0,Нет ...,ПОЖЕРТВОВАНИЕ,Банковские карты


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14031 entries, 0 to 14030
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  14031 non-null  object        
 1   sex                 12462 non-null  object        
 2   city                6737 non-null   object        
 3   e-mail_act          11073 non-null  datetime64[ns]
 4   pay_date            14031 non-null  datetime64[ns]
 5   pay_sum             14031 non-null  float64       
 6   recurrent           14031 non-null  object        
 7   purpose_of_payment  12762 non-null  object        
 8   payment_channel     14031 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.1+ MB


Заменим пропуски, просто на пустное значение. данные оставим близкими к реальности!

In [142]:
df.columns

Index(['id', 'sex', 'city', 'e-mail_act', 'pay_date', 'pay_sum', 'recurrent',
       'purpose_of_payment', 'payment_channel'],
      dtype='object')

In [143]:
columns = ['id', 
           'sex', 
           'city', 
           'recurrent',
           'purpose_of_payment', 
           'payment_channel',
           'e-mail_act']

for i in columns:
    df[i] = df[i].fillna('')
 
df.isna().sum()

id                       0
sex                      0
city                     0
e-mail_act            2958
pay_date                 0
pay_sum                  0
recurrent                0
purpose_of_payment       0
payment_channel          0
dtype: int64

In [144]:
df.head(10)

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment,payment_channel
0,AF575B85-917F-475D-8875-00055586E05B,,,NaT,2019-07-06 15:23:53,100.0,Нет ...,,SMS
1,229A9B43-3A15-4A48-8CB6-000EDB240D50,,,NaT,2017-07-18 09:01:24,500.0,Нет ...,,SMS
2,C6606486-05A8-4437-ACF9-00344E54FEE1,,,NaT,2019-10-21 18:31:00,300.0,Нет ...,,SMS
3,6ED86154-A688-4A11-868F-006209035413,Женский ...,,NaT,2021-04-13 00:00:00,500.0,Нет ...,Добровольное пожертвование. НДС не облагается,Прямые поступления на счет
4,46C41E20-EA56-4698-9C37-00692FD6FFE8,Женский ...,,NaT,2019-09-10 15:50:55,1042.0,Нет ...,ПОЖЕРТВОВАНИЕ,Банковские карты
5,8CBFF8AB-3153-473A-BE84-0079B873EA33,Мужской ...,,2022-07-18 17:36:39,2016-03-07 18:30:10,7000.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты
6,8CBFF8AB-3153-473A-BE84-0079B873EA33,Мужской ...,,2022-07-18 17:36:39,2016-07-07 18:30:00,7000.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты
7,8CBFF8AB-3153-473A-BE84-0079B873EA33,Мужской ...,,2022-07-18 17:36:39,2015-12-07 18:26:58,7000.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты
8,8CBFF8AB-3153-473A-BE84-0079B873EA33,Мужской ...,,2022-07-18 17:36:39,2016-04-07 18:30:07,7000.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты
9,8CBFF8AB-3153-473A-BE84-0079B873EA33,Мужской ...,,2022-07-18 17:36:39,2016-01-07 18:30:07,7000.0,Да ...,Пожертвование в фонд помощи детям-отказникам и...,Банковские карты


### Посмотрим на уникальные значния столбцов в датасете, вдруг что нужно изменить.

In [145]:
df['sex'].unique()

array(['',
       'Женский                                                                                                                                                                                                                                                   ',
       'Мужской                                                                                                                                                                                                                                                   '],
      dtype=object)

Видно пробелы после слов, удалим

In [146]:
df['city'].unique()

array(['', 'Брянск', 'Истра', 'Москва', 'Бирск', 'Казань', 'Ижевск',
       'Southall', 'ЗАСЕЧНОЕ', 'Самара', 'Frankfurt am Main',
       'Новосибирск', 'БЕРЕЗКА', 'Павловск', 'КИНГИСЕППСКИЙ',
       'Черноголовка', 'Краснодар', 'Якутск', 'Владивосток', 'Charleston',
       'Феодосия', 'Кемерово', 'Уфа', 'Ульяновск', 'Саров', 'БОРОВСКИЙ',
       'СТЕРЛИТАМАК', 'Нижний Новгород', 'Бухарест', 'Саратов', 'Lucca',
       'Северск', 'Усть-Илимск', 'Санкт-Петербург', 'Киров', 'Йошкар-Ола',
       'НОВОКУЙБЫШЕВСК', 'Тверь', 'Волхов', 'ЛЕНИНОГОРСК', 'Химки',
       'Омск', 'Заречный', 'Фрязино', 'ХАБАРОВСК', 'Сочи', 'Балашиха',
       'Астрахань', 'Арзамас', 'Королев', 'ТОБОЛЬСК', 'Denver',
       'АЛЬМЕТЬЕВСК', ' НАБЕРЕЖНЫЕ ЧЕЛНЫ ', 'ПИЖАНКА', 'Пермь',
       'Улан-Удэ', 'Челябинск', 'Чебоксары', 'Дубна', 'Щелково',
       'БАГДАРИН', 'Орёл', 'Ярославль', 'Brooklyn', 'Симферополь',
       'ПОЙКОВСКИЙ', 'Константиновск', 'Riga', 'Тольятти', 'Лыткарино',
       'Fort Lauderdale', 'Старый Оскол'

Тут всё в порядке

In [147]:
df['recurrent'].unique()

array(['Нет                                                                                                                                                                                                                                                       ',
       'Да                                                                                                                                                                                                                                                        '],
      dtype=object)

Также удалить пробелы после слов

In [148]:
df['purpose_of_payment'].unique()

array(['', 'Добровольное пожертвование. НДС не облагается',
       'ПОЖЕРТВОВАНИЕ',
       'Пожертвование в фонд помощи детям-отказникам и детям-сиротам "Бюро Добрых Дел"',
       'SMS bdd', 'Пожертвование на обучение Артёма',
       'ДОБРОВОЛЬНОЕ ПОЖЕРТВОВАНИЕ;Сумма 150 руб.;Комиссия 0 руб;Дата оплаты 13/12/2019;',
       'ДОБРОВОЛЬНОЕ ПОЖЕРТВОВАНИЕ;Сумма 200 руб.;Комиссия 0 руб;Дата оплаты 17/10/2020;',
       'ДОБРОВОЛЬНОЕ ПОЖЕРТВОВАНИЕ;Сумма 300 руб.;Комиссия 0 руб;Дата оплаты 26/12/2020;',
       'ДОБРОВОЛЬНОЕ ПОЖЕРТВОВАНИЕ;Сумма 300 руб.;Комиссия 0 руб;Дата оплаты 21/01/2021;',
       'Соц.квартиры в Костроме', 'ПОЖЕРТВОВАНИЕ  undefined',
       'Московские каникулы 2022',
       'На психологические консультации Артёму',
       'ПОЖЕРТВОВАНИЕ НА ПРОГРАММУ "Школа семьи""',
       'На обучение Карины и Алёны', 'На трудоустройство',
       'На обучение Дарьи', 'На обучение Елизаветы',
       'Кондитерская для\xa0Ксении', 'На обучение Полины',
       'ДОБРОВОЛЬНОЕ ПОЖЕРТВОВАНИЕ;смс 6

In [149]:
df['payment_channel'].unique()

array(['SMS  ', 'Прямые поступления на счет  ', 'Банковские карты  ',
       'Наличные в кассу  '], dtype=object)

In [150]:
# удалить пробелы посмле слов
df['sex'] = df['sex'].replace(r'\s+','',regex=True)
df['sex'].unique()

array(['', 'Женский', 'Мужской'], dtype=object)

In [151]:
# удалить пробелы посмле слов в реккурентности
df['recurrent'] = df['recurrent'].replace(r'\s+','',regex=True)
df['recurrent'].unique()

array(['Нет', 'Да'], dtype=object)

Столбцы с назначением платежа и каналом оплаты всё-таки надо категоризировать. Такая объёмная инфо ни к чему, да и выделяются определённые категории.

In [152]:
# приведём к нижнему регитсру
for column in df.select_dtypes("object").columns:
  df[column] = df[column].str.lower()
  
df.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment,payment_channel
0,af575b85-917f-475d-8875-00055586e05b,,,NaT,2019-07-06 15:23:53,100.0,нет,,sms
1,229a9b43-3a15-4a48-8cb6-000edb240d50,,,NaT,2017-07-18 09:01:24,500.0,нет,,sms
2,c6606486-05a8-4437-acf9-00344e54fee1,,,NaT,2019-10-21 18:31:00,300.0,нет,,sms
3,6ed86154-a688-4a11-868f-006209035413,женский,,NaT,2021-04-13 00:00:00,500.0,нет,добровольное пожертвование. ндс не облагается,прямые поступления на счет
4,46c41e20-ea56-4698-9c37-00692fd6ffe8,женский,,NaT,2019-09-10 15:50:55,1042.0,нет,пожертвование,банковские карты


In [153]:
df['purpose_of_payment'] = df['purpose_of_payment'].astype('str')

In [154]:
# назначение платежа
def rename (row):
  if 'пожертвование' in row or 'добровольное' in row:
    return 'пожертвование'
  if 'благотворит' in row:
    return 'благотворительность'
  if 'возврат' in row:
    return 'возврат'
  else:
    return 'иное'
  return row

df['purpose_of_payment_n'] = df['purpose_of_payment'].apply(rename)

In [155]:
df['purpose_of_payment_n'].value_counts()

пожертвование          11017
иное                    2934
благотворительность       56
возврат                   24
Name: purpose_of_payment_n, dtype: int64

Всё получилось, категоризировали данные. Сделаем тоже и со столбцом по каналам платежа.

In [156]:
# канал оплаты
def rename (row):
  if 'sms' in row:
    return 'смс'
  if 'поступл' in row:
    return 'прямые поступления'
  if 'карт' in row:
    return 'карта'
  if 'наличн' in row:
    return 'наличные'
  return row

df['payment_channel_n'] = df['payment_channel'].apply(rename)

In [157]:
df['payment_channel_n'].value_counts()

карта                 11204
смс                    1474
прямые поступления     1350
наличные                  3
Name: payment_channel_n, dtype: int64

In [158]:
df.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment,payment_channel,purpose_of_payment_n,payment_channel_n
0,af575b85-917f-475d-8875-00055586e05b,,,NaT,2019-07-06 15:23:53,100.0,нет,,sms,иное,смс
1,229a9b43-3a15-4a48-8cb6-000edb240d50,,,NaT,2017-07-18 09:01:24,500.0,нет,,sms,иное,смс
2,c6606486-05a8-4437-acf9-00344e54fee1,,,NaT,2019-10-21 18:31:00,300.0,нет,,sms,иное,смс
3,6ed86154-a688-4a11-868f-006209035413,женский,,NaT,2021-04-13 00:00:00,500.0,нет,добровольное пожертвование. ндс не облагается,прямые поступления на счет,пожертвование,прямые поступления
4,46c41e20-ea56-4698-9c37-00692fd6ffe8,женский,,NaT,2019-09-10 15:50:55,1042.0,нет,пожертвование,банковские карты,пожертвование,карта


Данные категоризировали. Теперь удалим лишние столбцы, от которых и делали категоризацию.

In [159]:
df = df.drop(columns=['purpose_of_payment',	'payment_channel'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14031 entries, 0 to 14030
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    14031 non-null  object        
 1   sex                   14031 non-null  object        
 2   city                  14031 non-null  object        
 3   e-mail_act            11073 non-null  datetime64[ns]
 4   pay_date              14031 non-null  datetime64[ns]
 5   pay_sum               14031 non-null  float64       
 6   recurrent             14031 non-null  object        
 7   purpose_of_payment_n  14031 non-null  object        
 8   payment_channel_n     14031 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.1+ MB


Посмотрим по года

In [160]:
df['pay_year'] = pd.DatetimeIndex(df['pay_date']).year

In [161]:
df.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment_n,payment_channel_n,pay_year
0,af575b85-917f-475d-8875-00055586e05b,,,NaT,2019-07-06 15:23:53,100.0,нет,иное,смс,2019
1,229a9b43-3a15-4a48-8cb6-000edb240d50,,,NaT,2017-07-18 09:01:24,500.0,нет,иное,смс,2017
2,c6606486-05a8-4437-acf9-00344e54fee1,,,NaT,2019-10-21 18:31:00,300.0,нет,иное,смс,2019
3,6ed86154-a688-4a11-868f-006209035413,женский,,NaT,2021-04-13 00:00:00,500.0,нет,пожертвование,прямые поступления,2021
4,46c41e20-ea56-4698-9c37-00692fd6ffe8,женский,,NaT,2019-09-10 15:50:55,1042.0,нет,пожертвование,карта,2019


In [162]:
df['pay_year'].value_counts()

2021    3547
2020    3195
2019    2455
2022    1739
2017    1252
2018     855
2016     761
2015     227
Name: pay_year, dtype: int64

Видно, что до 2019 года мало значений, значит самый активный период начинается с 2019 года. Его и возьмём в работу.

In [163]:
df = df.query("pay_year > 2018")
df['pay_year'].value_counts()

2021    3547
2020    3195
2019    2455
2022    1739
Name: pay_year, dtype: int64

Посмотрим на продолжительность действий каждого пользователя

In [164]:
df = df.sort_values(['id', 'pay_date'], ascending=[True, True])
df['life_time'] = df.groupby('id')['pay_date'].apply(lambda x: x.diff().fillna(pd.Timedelta(1, 'D')))
df['life_time']=df['life_time'].dt.days

In [165]:
df.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment_n,payment_channel_n,pay_year,life_time
11304,0001afdd-94e3-48f3-b5e7-d5539700425e,,,NaT,2019-04-26 19:19:24,100.0,нет,иное,смс,2019,1
4920,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-03-27 08:23:58,500.0,да,пожертвование,карта,2020,1
4903,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-04-29 08:24:29,500.0,да,пожертвование,карта,2020,33
4917,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-05-28 08:24:10,500.0,да,пожертвование,карта,2020,28
4909,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-07-27 08:24:02,500.0,да,пожертвование,карта,2020,59


In [166]:
df = df.drop(columns=['pay_year'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10936 entries, 11304 to 6556
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    10936 non-null  object        
 1   sex                   10936 non-null  object        
 2   city                  10936 non-null  object        
 3   e-mail_act            8406 non-null   datetime64[ns]
 4   pay_date              10936 non-null  datetime64[ns]
 5   pay_sum               10936 non-null  float64       
 6   recurrent             10936 non-null  object        
 7   purpose_of_payment_n  10936 non-null  object        
 8   payment_channel_n     10936 non-null  object        
 9   life_time             10936 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 939.8+ KB


### Для RFM-анализа потребуются определенные данные о каждом из жертвователей:

ID / канал оплаты / назначение платежа / дата платежа / реккурентность / сумма платежа - для их идентификации.

Давность (R) как количество дней с момента последней донации: Сколько дней назад была совершена их последняя донация? Вычтим дату последней покупки из сегодняшнего дня, чтобы рассчитать значение давности.

Частота (F) как общее количество донаций: Сколько раз клиент совершал донации? 

Деньги (М) как общая сумма потраченных денег: Сколько потратил этот клиент? Суммируем деньги от всех транзакций, чтобы получить значение M.

Чтобы извлечь эти значения, нам понадобятся только следующие столбцы из датасета.

'id', 'recurrent', 'pay_date' и 'pay_sum'.

In [167]:
df['pay_date'].max()

Timestamp('2022-07-28 19:22:15')

In [168]:
temp=['id', 'life_time', 'pay_date', 'pay_sum']
RFM_data=df[temp]
RFM_data.head()

,id,life_time,pay_date,pay_sum
11304,0001afdd-94e3-48f3-b5e7-d5539700425e,1,2019-04-26 19:19:24,100.0
4920,000711b2-2785-461b-bf5b-5cb7d2caba20,1,2020-03-27 08:23:58,500.0
4903,000711b2-2785-461b-bf5b-5cb7d2caba20,33,2020-04-29 08:24:29,500.0
4917,000711b2-2785-461b-bf5b-5cb7d2caba20,28,2020-05-28 08:24:10,500.0
4909,000711b2-2785-461b-bf5b-5cb7d2caba20,59,2020-07-27 08:24:02,500.0


Создание таблицы RFM

В датасете последняя донация была сделана 28 июля 2022 года, её и  использовуем в качестве даты для функции NOW, для расчёта давности.

In [171]:
NOW = dt.datetime(2022,7,28)

# RFM Table
RFM_table=RFM_data.groupby('id').agg({'pay_date': lambda x: (NOW - x.max()).days, # Recency
                                     'life_time': lambda x: x.sum(), 
                                     'pay_sum': lambda x: x.sum()}) # Monetary 

RFM_table.head()                         

,pay_date,life_time,pay_sum
id,,,
0001afdd-94e3-48f3-b5e7-d5539700425e,1188,1,100.0
000711b2-2785-461b-bf5b-5cb7d2caba20,30,814,10500.0
0015a100-2d8c-4262-a9b3-bb1461c0c8b8,365,1,400.0
006f71d7-dfcf-4e7a-b9ad-b4721bef4f7a,1187,1,200.0
008883d4-0182-4a4d-942c-4e55328a2c38,1051,1,1000.0


In [172]:
RFM_table.rename(columns={'pay_date': 'recency', 
                         'pay_sum': 'monetary_value'}, inplace=True)
RFM_table.head()

,recency,life_time,monetary_value
id,,,
0001afdd-94e3-48f3-b5e7-d5539700425e,1188,1,100.0
000711b2-2785-461b-bf5b-5cb7d2caba20,30,814,10500.0
0015a100-2d8c-4262-a9b3-bb1461c0c8b8,365,1,400.0
006f71d7-dfcf-4e7a-b9ad-b4721bef4f7a,1187,1,200.0
008883d4-0182-4a4d-942c-4e55328a2c38,1051,1,1000.0


In [173]:
RFM_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3078 entries, 0001afdd-94e3-48f3-b5e7-d5539700425e to ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   recency         3078 non-null   int64  
 1   life_time       3078 non-null   int64  
 2   monetary_value  3078 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 96.2+ KB


In [175]:
RFM_data_1 = RFM_data.groupby('id').agg({'pay_date': lambda x: x.count()})
RFM_data_1.head()

,pay_date
id,
0001afdd-94e3-48f3-b5e7-d5539700425e,1
000711b2-2785-461b-bf5b-5cb7d2caba20,21
0015a100-2d8c-4262-a9b3-bb1461c0c8b8,2
006f71d7-dfcf-4e7a-b9ad-b4721bef4f7a,1
008883d4-0182-4a4d-942c-4e55328a2c38,1


In [176]:
RFM_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3078 entries, 0001afdd-94e3-48f3-b5e7-d5539700425e to ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pay_date  3078 non-null   int64
dtypes: int64(1)
memory usage: 48.1+ KB


In [177]:
data_final = pd.concat([RFM_data_1, RFM_table], axis=1, sort=False)
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3078 entries, 0001afdd-94e3-48f3-b5e7-d5539700425e to ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pay_date        3078 non-null   int64  
 1   recency         3078 non-null   int64  
 2   life_time       3078 non-null   int64  
 3   monetary_value  3078 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 120.2+ KB


In [178]:
RFM_table = data_final

In [179]:
RFM_table['frequency'] = RFM_table['pay_date'] / RFM_table['life_time']
RFM_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3078 entries, 0001afdd-94e3-48f3-b5e7-d5539700425e to ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pay_date        3078 non-null   int64  
 1   recency         3078 non-null   int64  
 2   life_time       3078 non-null   int64  
 3   monetary_value  3078 non-null   float64
 4   frequency       3078 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 144.3+ KB


In [180]:
RFM_table = RFM_table.drop(columns=['life_time', 'pay_date'])
RFM_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3078 entries, 0001afdd-94e3-48f3-b5e7-d5539700425e to ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   recency         3078 non-null   int64  
 1   monetary_value  3078 non-null   float64
 2   frequency       3078 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 96.2+ KB


In [181]:
RFM_table = RFM_table.reindex(columns=['recency', 'frequency', 'monetary_value'])
RFM_table.head()

,recency,frequency,monetary_value
id,,,
0001afdd-94e3-48f3-b5e7-d5539700425e,1188,1.000000,100.0
000711b2-2785-461b-bf5b-5cb7d2caba20,30,0.025799,10500.0
0015a100-2d8c-4262-a9b3-bb1461c0c8b8,365,2.000000,400.0
006f71d7-dfcf-4e7a-b9ad-b4721bef4f7a,1187,1.000000,200.0
008883d4-0182-4a4d-942c-4e55328a2c38,1051,1.000000,1000.0


Расчитаем RFM. 

Для его расчета используем квинтили — составление четырех равных частей на основе доступных значений — для расчета показателя RFM.

In [182]:
quantiles = RFM_table.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,397.0,0.166667,150.0
0.50,847.5,1.000000,400.0
0.75,1099.0,1.000000,1337.5


In [183]:
# Converting quantiles to a dictionary, easier to use.
quantiles = quantiles.to_dict()

##  RFM Segmentation ----
RFM_Segment = RFM_table.copy()

# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def R_Class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FM_Class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
        
RFM_Segment['R_Quartile'] = RFM_Segment['recency'].apply(R_Class, args=('recency',quantiles,))
RFM_Segment['F_Quartile'] = RFM_Segment['frequency'].apply(FM_Class, args=('frequency',quantiles,))
RFM_Segment['M_Quartile'] = RFM_Segment['monetary_value'].apply(FM_Class, args=('monetary_value',quantiles,))
RFM_Segment['RFMClass'] = RFM_Segment.R_Quartile.map(str) \
                            + RFM_Segment.F_Quartile.map(str) \
                            + RFM_Segment.M_Quartile.map(str)

In [184]:
RFM_Segment.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id,,,,,,,
0001afdd-94e3-48f3-b5e7-d5539700425e,1188,1.000000,100.0,1,2,1,121
000711b2-2785-461b-bf5b-5cb7d2caba20,30,0.025799,10500.0,4,1,4,414
0015a100-2d8c-4262-a9b3-bb1461c0c8b8,365,2.000000,400.0,4,4,2,442
006f71d7-dfcf-4e7a-b9ad-b4721bef4f7a,1187,1.000000,200.0,1,2,2,122
008883d4-0182-4a4d-942c-4e55328a2c38,1051,1.000000,1000.0,2,2,3,223


In [185]:
RFM_Segment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3078 entries, 0001afdd-94e3-48f3-b5e7-d5539700425e to ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   recency         3078 non-null   int64  
 1   frequency       3078 non-null   float64
 2   monetary_value  3078 non-null   float64
 3   R_Quartile      3078 non-null   int64  
 4   F_Quartile      3078 non-null   int64  
 5   M_Quartile      3078 non-null   int64  
 6   RFMClass        3078 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 192.4+ KB


Вопрос: Кто лучшие жертвователи?

In [186]:
# RFMClass = 444
RFM_Segment[RFM_Segment['RFMClass']=='444'].sort_values('monetary_value', ascending=False).head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id,,,,,,,
0d9ea121-0e9a-44af-a9c1-35feddc4841c,263,3.5,70000.0,4,4,4,444
69a965e4-6c58-4e18-92d5-94e94a3971bd,241,3.0,50100.0,4,4,4,444
0cb8ac78-1325-43ed-ac95-041b6b4f8682,236,2.0,10000.0,4,4,4,444
8857abaa-229c-4061-98d0-2d3103575c21,62,3.0,10000.0,4,4,4,444
7680ff13-fea5-441c-afaf-5294ae2ff99d,209,3.0,4000.0,4,4,4,444


Вопрос: Какие жертвователи не активные?

In [187]:
RFM_Segment[RFM_Segment['R_Quartile'] <= 2 ].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id,,,,,,,
29c903a6-ae15-40fc-81fa-4f58d06f3139,959,1.000000,3000000.0,2,2,4,224
99d83c2d-f72a-4c58-9641-782996644f81,1246,1.000000,1000000.0,1,2,4,124
6ec0197e-4a86-47b6-a3ed-d9b6981bd4fe,927,1.000000,500000.0,2,2,4,224
c5bc87cd-9450-4a23-a699-62d15ffb3f8d,857,0.020725,322103.5,2,1,4,214
201ad296-15a8-4c82-b2d1-756d559f6dc0,1204,0.025316,268650.0,1,1,4,114


Вопрос: Кто такие потерянные жертвователи?

In [188]:
RFM_Segment[RFM_Segment['RFMClass']=='111'].sort_values('recency',ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id,,,,,,,
2a763751-2482-4a86-9db7-7255a7577b31,1201,0.044444,80.0,1,1,1,111
357fc296-b046-4f41-b9fe-07e389e7e5aa,1112,0.166667,150.0,1,1,1,111


Вопрос: Кто такие активные жертвователи?

In [189]:
RFM_Segment[RFM_Segment['F_Quartile'] >= 3 ].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
id,,,,,,,
0d9ea121-0e9a-44af-a9c1-35feddc4841c,263,3.5,70000.0,4,4,4,444
eed5ec19-ebdb-425d-a73f-1b65cae71cb8,1000,12.0,60000.0,2,4,4,244
69a965e4-6c58-4e18-92d5-94e94a3971bd,241,3.0,50100.0,4,4,4,444
993648ac-d826-4b08-86d5-8815a95d30a5,830,2.0,15000.0,3,4,4,344
0cb8ac78-1325-43ed-ac95-041b6b4f8682,236,2.0,10000.0,4,4,4,444


**Таким образом, можно выделить 4 группы и обозначить их как 1, 2, 3, 4.**

1 - потерянные жертвователи

2 - жертвователи не активные

3 - активные жертвователи

4 - самые активные жертвователи

### Соединение таблиц для объединения и подготвка к ML

### подготовим маленькую табличку с классами

In [190]:
RFM_Segment_drop = RFM_Segment.drop(columns=['recency',	'frequency',	'monetary_value',	'R_Quartile',	'F_Quartile',	'M_Quartile'])
RFM_Segment_drop.head(2)

,RFMClass
id,
0001afdd-94e3-48f3-b5e7-d5539700425e,121
000711b2-2785-461b-bf5b-5cb7d2caba20,414


In [191]:
# урезанный датасетс индексами выведенными
RFM_Segment_drop = RFM_Segment_drop.sort_values(by = ['id'], ascending = False).reset_index(drop = False)
RFM_Segment_drop.head(2)

,id,RFMClass
0,ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7,411
1,ffb1c4fa-f950-4bb7-a1f2-9dd655298fff,121


In [192]:
RFM_Segment_drop['RFMClass'].unique()

array(['411', '121', '414', '321', '223', '221', '122', '123', '213',
       '313', '222', '323', '322', '412', '421', '211', '423', '413',
       '224', '342', '212', '214', '314', '324', '424', '124', '444',
       '422', '244', '312', '443', '344', '343', '242', '114', '311',
       '442', '341', '112', '113', '243', '441', '143', '111', '142'],
      dtype=object)

In [193]:
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('111','1')

RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('142','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('112','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('211','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('113','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('213','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('214','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('114','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('122','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('123','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('222','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('221','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('121','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('212','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('124','2')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('143','2')

RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('411','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('414','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('413','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('412','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('443','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('344','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('343','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('311','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('312','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('314','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('313','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('244','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('441','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('243','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('341','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('442','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('242','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('422','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('424','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('324','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('342','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('224','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('423','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('421','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('322','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('323','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('223','3')
RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('321','3')

RFM_Segment_drop['RFMClass'] = RFM_Segment_drop['RFMClass'].replace('444','4')


RFM_Segment_drop['RFMClass'].unique()

array(['3', '2', '4', '1'], dtype=object)

In [194]:
RFM_Segment_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3078 entries, 0 to 3077
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3078 non-null   object
 1   RFMClass  3078 non-null   object
dtypes: object(2)
memory usage: 48.2+ KB


In [195]:
RFM_Segment_drop.head()

,id,RFMClass
0,ffccba23-89ca-4a22-bd8a-7cad2d9eb7f7,3
1,ffb1c4fa-f950-4bb7-a1f2-9dd655298fff,2
2,ffa30123-a516-44a7-b030-f83ed369b645,3
3,ff946fd1-eef8-4541-a906-bae64527aaa5,2
4,ff8daa4d-f7de-490a-aa62-d982fe587e4d,3


проверили датасет, есть все классы

In [196]:
# соединяем таблички
df_lm = pd.merge(df, RFM_Segment_drop, on=['id'], how='left') 
df_lm.head(10)

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment_n,payment_channel_n,life_time,RFMClass
0,0001afdd-94e3-48f3-b5e7-d5539700425e,,,NaT,2019-04-26 19:19:24,100.0,нет,иное,смс,1,2
1,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-03-27 08:23:58,500.0,да,пожертвование,карта,1,3
2,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-04-29 08:24:29,500.0,да,пожертвование,карта,33,3
3,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-05-28 08:24:10,500.0,да,пожертвование,карта,28,3
4,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-07-27 08:24:02,500.0,да,пожертвование,карта,59,3
5,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-08-27 08:24:04,500.0,да,пожертвование,карта,31,3
6,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-10-27 08:24:17,500.0,да,пожертвование,карта,61,3
7,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-11-27 08:24:03,500.0,да,пожертвование,карта,30,3
8,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-12-27 08:24:03,500.0,да,пожертвование,карта,30,3
9,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2021-02-28 08:24:15,500.0,да,пожертвование,карта,63,3


проверим, что самый лучший класс - 444 попал в таблицу и ничего не убежало.

In [197]:
df_lm.query("RFMClass == '4'").head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment_n,payment_channel_n,life_time,RFMClass
525,0cb8ac78-1325-43ed-ac95-041b6b4f8682,мужской,москва,2021-12-13 16:08:33,2021-12-03 17:41:32,5000.0,нет,иное,карта,1,4
526,0cb8ac78-1325-43ed-ac95-041b6b4f8682,мужской,москва,2021-12-13 16:08:33,2021-12-03 17:52:18,5000.0,нет,пожертвование,карта,0,4
572,0d9ea121-0e9a-44af-a9c1-35feddc4841c,мужской,fort lauderdale,2022-07-18 18:47:53,2021-11-05 18:26:59,10000.0,нет,иное,карта,1,4
573,0d9ea121-0e9a-44af-a9c1-35feddc4841c,мужской,fort lauderdale,2022-07-18 18:47:53,2021-11-05 18:29:48,10000.0,нет,иное,карта,0,4
574,0d9ea121-0e9a-44af-a9c1-35feddc4841c,мужской,fort lauderdale,2022-07-18 18:47:53,2021-11-05 18:37:50,10000.0,нет,иное,карта,0,4


In [198]:
# проверим пропуски
df_lm.isna().sum()

id                         0
sex                        0
city                       0
e-mail_act              2530
pay_date                   0
pay_sum                    0
recurrent                  0
purpose_of_payment_n       0
payment_channel_n          0
life_time                  0
RFMClass                   0
dtype: int64

In [199]:
# смотрим общую инфо
df_lm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10936 entries, 0 to 10935
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    10936 non-null  object        
 1   sex                   10936 non-null  object        
 2   city                  10936 non-null  object        
 3   e-mail_act            8406 non-null   datetime64[ns]
 4   pay_date              10936 non-null  datetime64[ns]
 5   pay_sum               10936 non-null  float64       
 6   recurrent             10936 non-null  object        
 7   purpose_of_payment_n  10936 non-null  object        
 8   payment_channel_n     10936 non-null  object        
 9   life_time             10936 non-null  int64         
 10  RFMClass              10936 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 1.0+ MB


## Кодирование

In [200]:
df_lm.columns

Index(['id', 'sex', 'city', 'e-mail_act', 'pay_date', 'pay_sum', 'recurrent',
       'purpose_of_payment_n', 'payment_channel_n', 'life_time', 'RFMClass'],
      dtype='object')

In [201]:
columns = ['sex', 'city', 'e-mail_act', 'pay_date', 'recurrent', 'purpose_of_payment_n', 'payment_channel_n']

for i in columns:
   df_lm[i] = (df_lm[i]).astype('category')

df_lm['RFMClass'] = df_lm['RFMClass'].astype('int')

df_lm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10936 entries, 0 to 10935
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    10936 non-null  object  
 1   sex                   10936 non-null  category
 2   city                  10936 non-null  category
 3   e-mail_act            8406 non-null   category
 4   pay_date              10936 non-null  category
 5   pay_sum               10936 non-null  float64 
 6   recurrent             10936 non-null  category
 7   purpose_of_payment_n  10936 non-null  category
 8   payment_channel_n     10936 non-null  category
 9   life_time             10936 non-null  int64   
 10  RFMClass              10936 non-null  int64   
dtypes: category(7), float64(1), int64(2), object(1)
memory usage: 903.6+ KB


In [202]:
df_lm.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment_n,payment_channel_n,life_time,RFMClass
0,0001afdd-94e3-48f3-b5e7-d5539700425e,,,NaT,2019-04-26 19:19:24,100.0,нет,иное,смс,1,2
1,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-03-27 08:23:58,500.0,да,пожертвование,карта,1,3
2,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-04-29 08:24:29,500.0,да,пожертвование,карта,33,3
3,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-05-28 08:24:10,500.0,да,пожертвование,карта,28,3
4,000711b2-2785-461b-bf5b-5cb7d2caba20,женский,,2022-07-27 10:16:14,2020-07-27 08:24:02,500.0,да,пожертвование,карта,59,3


In [203]:
columns = ['sex', 'city', 'e-mail_act', 'pay_date', 'recurrent', 'purpose_of_payment_n', 'payment_channel_n']

for i in columns:
   df_lm[i] = (df_lm[i]).cat.codes

df_lm.head()

,id,sex,city,e-mail_act,pay_date,pay_sum,recurrent,purpose_of_payment_n,payment_channel_n,life_time,RFMClass
0,0001afdd-94e3-48f3-b5e7-d5539700425e,0,0,-1,432,100.0,1,2,3,1,2
1,000711b2-2785-461b-bf5b-5cb7d2caba20,1,0,535,2914,500.0,0,3,0,1,3
2,000711b2-2785-461b-bf5b-5cb7d2caba20,1,0,535,3251,500.0,0,3,0,33,3
3,000711b2-2785-461b-bf5b-5cb7d2caba20,1,0,535,3517,500.0,0,3,0,28,3
4,000711b2-2785-461b-bf5b-5cb7d2caba20,1,0,535,4028,500.0,0,3,0,59,3


In [204]:
df_lm.dtypes

id                       object
sex                        int8
city                      int16
e-mail_act                int16
pay_date                  int16
pay_sum                 float64
recurrent                  int8
purpose_of_payment_n       int8
payment_channel_n          int8
life_time                 int64
RFMClass                  int64
dtype: object

Подготовим данные для обучения

In [205]:
# удалим айди
df_lm = df_lm.drop(columns=['id'], axis=1)

## Обучение

***Оценивать качество моделей будем:***

accuracy - чем выше, тем лучше (равная 0.5714, означает, что модель с точностью 57,14% делает верный прогноз)

готовим выборки

In [206]:
X = df_lm.drop(['RFMClass'], axis = 1) 
y = df_lm['RFMClass']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_st = scaler.fit_transform(X_train)
X_test_st = scaler.transform(X_test)

### LogisticRegression

In [207]:
%%timeit

lr = LogisticRegression() # объявляем модель

lr_params = {'fit_intercept':[True,False], 
             'max_iter': [1,10]} # объявляем параметры

scorers = make_scorer(accuracy_score)

lr_grid = GridSearchCV(lr, 
                       lr_params, 
                       cv=5, 
                       refit=False,
                       scoring=scorers) # объявляем подбор гиперпараметров + скоринг

lr_grid.fit(X_train_st, y_train) # учим модель

display("accuracy составляет", lr_grid.best_score_)
display('Наилучшие подобранные параметры', lr_grid.best_params_)

'accuracy составляет'

0.9434155027362575

'Наилучшие подобранные параметры'

{'fit_intercept': True, 'max_iter': 10}

'accuracy составляет'

...

'accuracy составляет'

0.9434155027362575

'Наилучшие подобранные параметры'

{'fit_intercept': True, 'max_iter': 10}

1 loop, best of 5: 447 ms per loop


### RandomForestClassifie

In [208]:
%%timeit

clf = RandomForestClassifier()

parametrs = {'n_estimators': range (2, 10),
            'max_depth': range (1,10)}

scorers=make_scorer(accuracy_score)

grid = GridSearchCV(clf, parametrs, cv=3, scoring=scorers)
grid.fit(X_train, y_train)

display('Best parametrs:', grid.best_params_)
display('Accurascy score:', grid.best_score_)

'Best parametrs:'

{'max_depth': 8, 'n_estimators': 9}

'Accurascy score:'

0.9726794695930497

'Best parametrs:'

...

'Best parametrs:'

{'max_depth': 9, 'n_estimators': 9}

'Accurascy score:'

0.9738225880201189

1 loop, best of 5: 5.75 s per loop


### CatBoostClassifier

In [209]:
%%timeit

clf = CatBoostClassifier()

parametrs = {'depth': [1, 3]}

scorers=make_scorer(accuracy_score)

grid = GridSearchCV(clf, parametrs, cv=3, scoring=scorers)
grid.fit(X_train, y_train)

display('Best parametrs:', grid.best_params_)
display('Accurascy score:', grid.best_score_)

Выходные данные были обрезаны до нескольких последних строк (5000).
4:	learn: 0.7549141	total: 12.2ms	remaining: 2.43s
5:	learn: 0.6894013	total: 14.5ms	remaining: 2.4s
6:	learn: 0.6338492	total: 16.8ms	remaining: 2.39s
7:	learn: 0.5859012	total: 19.1ms	remaining: 2.37s
8:	learn: 0.5442979	total: 21.4ms	remaining: 2.36s
9:	learn: 0.5070219	total: 23.7ms	remaining: 2.35s
10:	learn: 0.4750801	total: 26.1ms	remaining: 2.34s
11:	learn: 0.4470135	total: 28.4ms	remaining: 2.34s
12:	learn: 0.4216339	total: 30.8ms	remaining: 2.34s
13:	learn: 0.3993596	total: 33.1ms	remaining: 2.33s
14:	learn: 0.3795456	total: 35.6ms	remaining: 2.33s
15:	learn: 0.3588301	total: 37.9ms	remaining: 2.33s
16:	learn: 0.3406693	total: 40.3ms	remaining: 2.33s
17:	learn: 0.3261538	total: 42.9ms	remaining: 2.34s
18:	learn: 0.3131837	total: 45.4ms	remaining: 2.34s
19:	learn: 0.3012576	total: 47.7ms	remaining: 2.34s
20:	learn: 0.2903693	total: 50ms	remaining: 2.33s
21:	learn: 0.2783031	total: 52.3ms	remaining: 2.32s
22:	l

'Best parametrs:'

{'depth': 3}

'Accurascy score:'

0.9739368998628258

...

'Best parametrs:'

{'depth': 3}

'Accurascy score:'

0.9739368998628258

1 loop, best of 5: 23.4 s per loop


### LGBMClassifier

In [210]:
%%timeit

clf = LGBMClassifier()

parametrs = {'n_jobs': range(1, 4), 
             'learning_rate':[0.01, 0.04]}

scorers=make_scorer(accuracy_score)

grid = GridSearchCV(clf, parametrs, cv=3, scoring=scorers)
grid.fit(X_train, y_train)

display('Best parametrs:', grid.best_params_)
display('Accurascy score:', grid.best_score_)

'Best parametrs:'

{'learning_rate': 0.04, 'n_jobs': 1}

'Accurascy score:'

0.9767946959304984

'Best parametrs:'

...

'Best parametrs:'

{'learning_rate': 0.04, 'n_jobs': 1}

'Accurascy score:'

0.9767946959304984

1 loop, best of 5: 14.9 s per loop


**Выводы**

In [211]:
#сделаем табличку, чтобы удобнее было делать выводы
index =['Линейная регрессия', 
        'Случайный лес', 
        'CatBoost',
        'LightGBM']

accuracy = ['0.943',
            '0.973', 
            '0.973',
            '0.976'] 
 
time_sec = ['0.44',
            '5.75', 
            '23.4',
            '14.9'] 

list_tuples = list(zip(accuracy, time_sec)) 
 
dframe = pd.DataFrame(list_tuples, index = index, columns=['accuracy', 'time_sec']) 
 
display(dframe)

,accuracy,time_sec
Линейная регрессия,0.943,0.44
Случайный лес,0.973,5.75
CatBoost,0.973,23.4
LightGBM,0.976,14.9


По метрике CatBoost & LGBM почти одинаковые, но по времени LGBM выигрывает.

## Тестирование лучшей модели

### LGBMClassifier

In [212]:
%%timeit

clf = LGBMClassifier(n_jobs = 1, learning_rate = 0.04)
clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test) 
acc = accuracy_score(y_test, clf_predictions)

display('Accurascy score:', acc)

'Accurascy score:'

0.9803473491773309

'Accurascy score:'

0.9803473491773309

'Accurascy score:'

...

'Accurascy score:'

0.9803473491773309

'Accurascy score:'

0.9803473491773309

1 loop, best of 5: 624 ms per loop


Результат получился - 98% совпадений.